# UW-Madison GI Tract Image Segmentation - Fardella, Tortorici

## Descrizione del problema

Nel 2019, si stima che in tutto il mondo siano state diagnosticate 5 milioni di persone con un **tumore del tratto gastrointestinale**. Di questi pazienti, circa la metà può essere sottoposta a radioterapia, di solito somministrata per 10-15 minuti al giorno per 1-6 settimane. 

Gli oncologi radioterapisti cercano di somministrare dosi elevate di radiazioni utilizzando fasci di raggi X puntati sui tumori, evitando lo stomaco e l'intestino. Con le nuove tecnologie, come i **sistemi integrati di risonanza magnetica e acceleratore lineare**, noti anche come *MR-Linac*, gli oncologi sono in grado di visualizzare la posizione giornaliera del tumore e dell'intestino, che può variare di giorno in giorno. 

In queste scansioni, gli oncologi radioterapisti devono delineare **manualmente** la posizione dello stomaco e dell'intestino per regolare la direzione dei fasci di raggi X in modo da **aumentare** la dose erogata al tumore ed **evitare** lo stomaco e l'intestino. Si tratta di un processo lungo e laborioso che può prolungare i trattamenti da 15 minuti al giorno a un'ora al giorno, il che può essere difficile da tollerare per i pazienti, a meno che *il deep learning non aiuti ad automatizzare il processo di segmentazione*. Un metodo per segmentare lo stomaco e l'intestino renderebbe i trattamenti molto più veloci e consentirebbe a un maggior numero di pazienti di ricevere un trattamento più efficace.

L'**UW-Madison Carbone Cancer Center** è un pioniere della radioterapia basata sulla risonanza magnetica e dal 2015 tratta i pazienti con *radioterapia guidata dalla risonanza magnetica in base alla loro anatomia quotidiana*.

L'obiettivo è creare un modello per segmentare automaticamente lo stomaco e l'intestino nelle scansioni MRI, basandosi su un data set di queste scansioni per trovare soluzioni creative di deep learning che aiutino i pazienti oncologici a ricevere cure migliori.

![Tumore allo stomaco](./pics/pic_001.jpeg "Tumore allo stomaco")

In questa figura, il tumore (linea spessa rosa) è vicino allo stomaco (linea spessa rossa). Al tumore vengono indirizzate dosi elevate di radiazioni, evitando lo stomaco. I livelli di dose sono rappresentati dall'arcobaleno dei contorni, con le dosi più alte rappresentate dal rosso e quelle più basse dal verde.

Il cancro è già abbastanza pesante, ma la soluzione al problema permetterebbe agli oncologi radioterapisti di somministrare in modo sicuro dosi più elevate di radiazioni ai tumori, evitando lo stomaco e l'intestino. Questo renderà più veloci i trattamenti quotidiani dei pazienti oncologici e consentirà loro di ottenere cure più efficaci con meno effetti collaterali e un migliore controllo del cancro a lungo termine.

### *Valutazione*

La bontà della nostra soluzione viene valutata tramite il coefficiente di Dice e la distanza 3D di Hausdorff. 

Il **coefficiente di Dice** può essere utilizzato per confrontare il rapporto tra i pixel di una segmentazione prevista e la corrispondente *ground truth*, secondo la seguente legge:

$$
D_c = \begin{cases} 
0 && X\wedge Y vuoti \\
2\cdot \frac{|X\cap Y|}{|X|+|Y|} && altrimenti
\end{cases}
$$

dove $X$ è l'insieme dei *pixel previsti* e $Y$ è la *ground truth*. 

La **distanza di Hausdorff** è un metodo per calcolare la distanza tra due oggetti di segmentazione A e B, considerando il **punto più lontano di A** dal **punto più vicino di B**.
Per applicare Hausdorff in 3D, costruiamo volumi 3D, combinando ogni segmentazione 2D con una profondità di slice come coordinata Z, per poi calcolare la distanza (ai fini della competizione la profondità di slice è settata a 0.1. Le posizioni previste/attese dei pixel sono normalizzate con la dimensione dell’immagine, in modo da ottenere un punteggio in $\left[ 0-1\right]$.

Successivamente, vengono combinate le due metriche, con un peso di $0.4$ per il coefficiente Dice e un peso di $0.6$ per la distanza di Hausdorff.

### *Dataset*

Le annotazioni di addestramento sono fornite come **maschere codificate in RLE** e le immagini sono in formato `.png` a 16 bit e in **scala di grigi**.

Ogni caso nel dataset è rappresentato da *più serie di slice di scansione* (ogni serie è identificata dal giorno in cui è stata effettuata la scansione). Alcuni dati sono divisi per tempo (i primi giorni di un caso sono nel training set, i giorni successivi sono nel test set), mentre altri sono divisi per caso - l'intero caso è nel training set o nel test set. 

L'obiettivo che ci si pone (sulla base della [competizione](https://www.kaggle.com/competitions/uw-madison-gi-tract-image-segmentation) da cui è tratto il problema) è quello di essere in grado di generalizzare sia casi parzialmente visti che **interamente non visti**.

Per la definizione dei singoli set, non avendo a disposizione un test set, abbiamo adottato una suddivisione 90%-10% dei dati in nostro possesso tra training set e test set.

## Progettazione

`-- TO-DO --`

### 1. *Data Preparation*

In [26]:
import os
import pandas as pd

BASE_DIR = '../.venv/fardella_tortorici/BD-Image-Seg-Dataset'
TRAIN_DIR = os.path.join(BASE_DIR, 'train')
TRAIN_CSV = os.path.join(BASE_DIR, 'train.csv')

```python
# Si risolve interamente con l'approccio trovato sul web?
```
Come primo passo, abbiamo provveduto a importare i set necessari al training e al testing, sfruttando la libreria Pandas, nello specifico la funzione `read_csv(filepath_or_buffer)`, cui abbiamo passato in input il percorso al file `submission.csv`, per ottenere un dataframe che, se vuoto, comporta l'abilitazione di una modalità *debug*.

In [27]:
TEST_DIR = os.path.join(BASE_DIR, 'test')
SUBMISSION_CSV = os.path.join(BASE_DIR, 'submission.csv')

submission_df = pd.read_csv(SUBMISSION_CSV)
if not len(submission_df):
    debug = True
    submission_df = pd.read_csv(SUBMISSION_CSV)
    submission_df = submission_df.drop(columns=['class','predicted']).drop_duplicates()
else:
    debug = False
    submission_df = submission_df.drop(columns=['class','predicted']).drop_duplicates()

Successivamente, abbiamo estratto da ogni riga del dataframe appena importato dei metadati di interesse attraverso la funzione utility `get_metadata(row)`, nello specifico il numero del caso (`case`), il giorno di monitoraggio (`day`) e la sezione della scansione di interesse (`slice`).

In [28]:
def get_metadata(row):
    """Funzione usata per estrarre dei metadati da una riga rappresentante uno
    specifico frammento di uno specifico caso in uno specifico giorno

    Args:
        row (list[str]): la riga del dataframe da cui estrarre i metadati

    Returns:
        list[str]: la riga passata in input da cui sono stati estratti i metadati
    """
    
    data = row['id'].split('_')
    case = int(data[0].replace('case',''))
    day = int(data[1].replace('day',''))
    slice_ = int(data[-1])
    row['case'] = case
    row['day'] = day
    row['slice'] = slice_
    return row

submission_df = submission_df.apply(lambda x: get_metadata(x),axis=1)

Ottenendo una rappresentazione del dataframe delle submission così fatto.

In [29]:
print("\n--- SUBMISSION DATAFRAME ---\n")
submission_df.head(5)


--- SUBMISSION DATAFRAME ---



,id,case,day,slice
0,case2_day1_slice_0001,2,1,1
3,case2_day1_slice_0002,2,1,2
6,case2_day1_slice_0003,2,1,3
9,case2_day1_slice_0004,2,1,4
12,case2_day1_slice_0005,2,1,5


Dopodiché, abbiamo importato i percorsi di tutte le scansioni del training set, attraverso la funzione `glob`, e ne abbiamo ricavato un secondo dataframe, la cui unica colonna è il percorso dell'immagine stessa

In [30]:
from glob import glob

paths = glob(os.path.join(TRAIN_DIR, '**', '*png'), recursive=True)

path_df = pd.DataFrame(paths, columns=['image_path'])

Attraverso una seconda funzione ausiliaria `path2info(row)`, abbiamo modellato il dataframe del training set in modo da mostrare caratteristiche di interesse come le dimensioni dell'immagine, il caso specifico, il giorno di monitoraggio e la porzione di scansione di interesse, oltre a un `image_id` univoco per l'immagine

In [31]:
def path2info(row):
    path = row['image_path']
    # print(path)
    path = path.replace("\\", "/")
    data = path.split('/')
    slice_ = int(data[-1].split('_')[1])
    case = int(data[-3].split('_')[0].replace('case',''))
    day = int(data[-3].split('_')[1].replace('day',''))
    width = int(data[-1].split('_')[2])
    height = int(data[-1].split('_')[3])
    row['image_id'] = "case%s_day%s_slice_%s" % (case, day, str(slice_).zfill(4))
    # print(row['image_id'])
    row['height'] = height
    row['width'] = width
    row['case'] = case
    row['day'] = day
    row['slice'] = slice_
    return row

path_df = path_df.apply(lambda x: path2info(x), axis=1)

Ottenendo un dataframe dei percorsi così fatto:

In [32]:
print("\n--- PATHS DATAFRAME ---\n")
path_df.head(5)


--- PATHS DATAFRAME ---



,image_path,image_id,height,width,case,day,slice
0,../.venv/fardella_tortorici/BD-Image-Seg-Datas...,case22_day0_slice_0087,266,266,22,0,87
1,../.venv/fardella_tortorici/BD-Image-Seg-Datas...,case22_day0_slice_0002,266,266,22,0,2
2,../.venv/fardella_tortorici/BD-Image-Seg-Datas...,case22_day0_slice_0080,266,266,22,0,80
3,../.venv/fardella_tortorici/BD-Image-Seg-Datas...,case22_day0_slice_0029,266,266,22,0,29
4,../.venv/fardella_tortorici/BD-Image-Seg-Datas...,case22_day0_slice_0125,266,266,22,0,125


Infine, abbiamo effettuato l'unione tra i due dataframe definiti in precedenza, sfruttando l'uguaglianza sui valori di `case`, `day` e `slice`, ottenendo un dataframe così fatto:

In [33]:
path_df = path_df.merge(submission_df, on=['case','day','slice'], how='left')

print('\n--- MERGED PATHS DATAFRAME ---\n')
path_df.head(5)


--- MERGED PATHS DATAFRAME ---



,image_path,image_id,height,width,case,day,slice,id
0,../.venv/fardella_tortorici/BD-Image-Seg-Datas...,case22_day0_slice_0087,266,266,22,0,87,NaN
1,../.venv/fardella_tortorici/BD-Image-Seg-Datas...,case22_day0_slice_0002,266,266,22,0,2,NaN
2,../.venv/fardella_tortorici/BD-Image-Seg-Datas...,case22_day0_slice_0080,266,266,22,0,80,NaN
3,../.venv/fardella_tortorici/BD-Image-Seg-Datas...,case22_day0_slice_0029,266,266,22,0,29,NaN
4,../.venv/fardella_tortorici/BD-Image-Seg-Datas...,case22_day0_slice_0125,266,266,22,0,125,NaN


In [ ]:
path_df.drop(labels='id')

#### 1. Features Extraction

#### 2. Data Cleaning

#### 3. Features Selection & Transformation

### 2. *Data Analysis*

```python
# Va fatta? Quante metriche potrebbero andare bene visto che siamo in due?
```

### 3. *Classificazione*

#### 1. Architettura Utilizzata

```python
# Potrebbe aver senso una rete convoluzionale?
```

#### 2. Implementazione

#### 3. Risultati